# Unteruschung der Extraktionsqualität

In [1]:
import os
import pandas as pd
from ddbapi import zp_pages

places = ["Köln"]

outdir = "data"
if not os.path.isdir(outdir):
    os.mkdir(outdir)
#Save all the datasets individually - so we have a checkpoint for the org. data
df_list = []
for place in places:
    df = zp_pages(
        publication_date='[1850-01-01T12:00:00Z TO 1980-12-31T12:00:00Z]', 
        place_of_distribution=place, 
        plainpagefulltext=["Heiratsgesuch"]
        )
    
    df_list.append(df)

df_final = pd.concat(df_list)
df_final.to_pickle("ddbapi_data_koeln.pkl")

https://api.deutsche-digitale-bibliothek.de/search/index/newspaper-issues/select?rows=1000&sort=id+ASC&q=type%3Apage+AND+publication_date%3A%22%5B1850-01-01T12%3A00%3A00Z%5C+TO%5C+1980-12-31T12%3A00%3A00Z%5D%22+AND+place_of_distribution%3A%22K%C3%B6ln%22+AND+%28plainpagefulltext%3AHeiratsgesuch%29&cursorMark=%2A
Getting 1000 of 2696
Getting 2000 of 2696
Getting 2696 of 2696
Got 2696 items.


In [8]:
import pandas as pd

df_final = pd.read_pickle("ddbapi_data_koeln.pkl")

In [9]:
from scripts.information_extractor_lib import InformationExtractor

information_extractor = InformationExtractor(df=df_final, page_id_colname="page_id", text_colname="plainpagefulltext", output_filename="out_df.csv")
ergebnis_df = information_extractor.extract_data_loop(max_n = 100)
print(ergebnis_df)

c:\Users\adrim\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<bound method InformationExtractor.set_config of <scripts.information_extractor_lib.InformationExtractor object at 0x000001E873CC1940>> is done!
<bound method InformationExtractor.load_model of <scripts.information_extractor_lib.InformationExtractor object at 0x000001E873CC1940>> is done!
<bound method InformationExtractor.load_data of <scripts.information_extractor_lib.InformationExtractor object at 0x000001E873CC1940>> is done!
{'content': ['Suche für meine Tochter , 18 Jahre , Stelle als angeh . Arbeiterin in kathol . Hause bei Familienanschl . in Boun oder Umgegend . Augebote u . A . K . A . 4368 .', "Heirats - Gesuche Bei mon aie Gerdprenderch nungen oder dgl . bezahlen . D . A . - Zee . Aufricht . Heiratsgesuch . Gut gebildeter , derm karc . Fleischerssohn mit angeneß . Neußern sucht , weil seit Vo tun des Krieges im Jeldo und ihm daher an passenden Damenbekanntschaft fehlt . in ein näheres Verhältnis mit junger , hübscher , kath . Flei chersrochter ( 19 — 24 Jahr . ) in Briefwec

# Unnesting the Data - because per row multiple results are saved

In [10]:
def unnest_data(ergebnis_df:pd.DataFrame) -> pd.DataFrame:
    erg_list = []
    for i in range(len(ergebnis_df)):
        temp_list = []
        row = ergebnis_df.iloc[i,:]
        for t in row["text"]:
            temp_list.append(t)
        n = len(temp_list)
        df_temp = pd.DataFrame(data={"page_id":[row["page_id"]]*n, "text":temp_list,"time":[row["time"]]*n})
        erg_list.append(df_temp)
    df_unnested = pd.concat(erg_list)
    return df_unnested.reset_index().filter(["page_id", "text", "time"])



In [11]:
full_data = unnest_data(ergebnis_df)

In [12]:
full_data

,page_id,text,time
0,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Suche für meine Tochter , 18 Jahre , Stelle al...",2025-06-11_12_11
1,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...,2025-06-11_12_11
2,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Junger Mann . 2 J hre , kath . , sucht aufrich...",2025-06-11_12_11
3,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Zwei junge Herren , 19 # . 21 Jahre , suchen d...",2025-06-11_12_11
4,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Tehal nach Brükcnes . Gewissenhafter deutscher...,2025-06-11_12_11
...,...,...,...
443,3A3Y7H67WQGOWZ55X4SWY63YD36HAF7A-ALTO10126372_...,"Aufrichtiges Heiratsgesuch . Alleins , Jungges...",2025-06-11_12_11
444,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,Heirat . Mehrere reiche Herren in ho sichern L...,2025-06-11_12_11
445,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,"Eabrikant der Eisenbranche , 28 Jahre alt , ka...",2025-06-11_12_11
446,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,Aufrichtiges Heiratsgesuch oder Einheirat . ic...,2025-06-11_12_11


In [13]:
unique_page_ids = full_data[full_data.filter(["page_id"]).duplicated() == False]["page_id"].to_list()

## Berechnung der Accuracy der Informationsextraction 

Es wird für jeden Full-Text für Berechnet ob die entnommenen Kontaktanzeigen 1:1 genau so im Volltext enthalten sind wie sie von der GeminiAPI entnommen werden. Es wird ein über die Anzahl der entnommenen Anzeigen gewichteter Mittelwert der Accuracy berechnet.
  

In [14]:
n = len(full_data)
total_extracted = 0
weighted_perfect_extraction_ratio = 0
for i in range(len(unique_page_ids)):

    curr_full_text = df_final[df_final.page_id == unique_page_ids[i]].plainpagefulltext.tolist()[0]
    exctracted = full_data[full_data.page_id == unique_page_ids[i]].text.tolist()
    perfect_extraction_ratio = sum([e in curr_full_text for e in exctracted])/len(exctracted)
    total_extracted += sum([e in curr_full_text for e in exctracted])
    weighted_perfect_extraction_ratio += perfect_extraction_ratio*(len(exctracted)/n)



    #print(f"Extraction Ratio {perfect_extraction_ratio}, Len of extracted {len(exctracted)} ")

print(f"Total perfect_extraction_ratio weighted {weighted_perfect_extraction_ratio}")
print(f"Total extraction rate: {total_extracted/n}")






Total perfect_extraction_ratio weighted 0.7879464285714285
Total extraction rate: 0.7879464285714286


# Untersuchen ob es kleinere Fehler gibt, die nicht den Sinn/Struktur verändern.

Prüfen ob der String mit einer annähernden Gleichheit im Originaltext vorhanden ist.

In [15]:
df_red = df_final.filter(["page_id", "plainpagefulltext"])

dat_joined = pd.merge(full_data, df_red, on="page_id", how="left")
dat_joined

,page_id,text,time,plainpagefulltext
0,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Suche für meine Tochter , 18 Jahre , Stelle al...",2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
1,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...,2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
2,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Junger Mann . 2 J hre , kath . , sucht aufrich...",2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
3,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Zwei junge Herren , 19 # . 21 Jahre , suchen d...",2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
4,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Tehal nach Brükcnes . Gewissenhafter deutscher...,2025-06-11_12_11,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
...,...,...,...,...
443,3A3Y7H67WQGOWZ55X4SWY63YD36HAF7A-ALTO10126372_...,"Aufrichtiges Heiratsgesuch . Alleins , Jungges...",2025-06-11_12_11,1 A M bih F 14 A 4 — * Ihre Kleider = Wünsche ...
444,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,Heirat . Mehrere reiche Herren in ho sichern L...,2025-06-11_12_11,Wir suchen per sofort event . 1 . Oktober Spez...
445,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,"Eabrikant der Eisenbranche , 28 Jahre alt , ka...",2025-06-11_12_11,Wir suchen per sofort event . 1 . Oktober Spez...
446,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,Aufrichtiges Heiratsgesuch oder Einheirat . ic...,2025-06-11_12_11,Wir suchen per sofort event . 1 . Oktober Spez...


In [16]:
not_correct_extracted_df = dat_joined[dat_joined.apply(lambda row: row["text"] in row["plainpagefulltext"], axis = 1) == False]
not_correct_extracted_df

,page_id,text,time,plainpagefulltext
1,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...,2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
3,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Zwei junge Herren , 19 # . 21 Jahre , suchen d...",2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
4,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Tehal nach Brükcnes . Gewissenhafter deutscher...,2025-06-11_12_11,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
9,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"teiraf 27jährig . Herr , ev . , aus erster rhe...",2025-06-11_12_11,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
16,245BHDGA6W5OOYPCWEGLHABY6LM7YNXP-ALTO7758526_D...,"Heirats - Gesuch. Ein Kaufm . ( Ehrenmann ) , ...",2025-06-11_12_11,"Soennecken - Federn Das beste , was die Schrei..."
...,...,...,...,...
419,373GCHWZ25DT4GNR5J44JCNUHG3GVKQX-ALTO8385016_D...,"Glückliche Heirat ! ger Herr , evgl. , großé F...",2025-06-11_12_11,An die Stammaktionäre der Howaldtswerke . Auf ...
422,373GCHWZ25DT4GNR5J44JCNUHG3GVKQX-ALTO8385016_D...,"nchmster , diskretester Weise mi reichen , jun...",2025-06-11_12_11,An die Stammaktionäre der Howaldtswerke . Auf ...
429,37KC2RXMJ5E32B2WKEU7EOV4SF3KCQ5K-ALTO7744027_D...,Heirat . Jetzt besonders erquickende und stärk...,2025-06-11_12_11,Teder ! Bauers Reform Feuer - Annihilator ist ...
432,37KC2RXMJ5E32B2WKEU7EOV4SF3KCQ5K-ALTO7744027_D...,"ucte junge , urklch hübsche W Dame mit 58 . Ve...",2025-06-11_12_11,Teder ! Bauers Reform Feuer - Annihilator ist ...


In [17]:
dat_joined[dat_joined.apply(lambda row: row["text"] in row["plainpagefulltext"], axis=1)]

,page_id,text,time,plainpagefulltext
0,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Suche für meine Tochter , 18 Jahre , Stelle al...",2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
2,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Junger Mann . 2 J hre , kath . , sucht aufrich...",2025-06-11_12_11,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
5,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Sr Rittergutsbesitzers - Sohn 30 J . alt , Erb...",2025-06-11_12_11,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
6,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Heiratsgesuch . Ingenieur , 31 Jahre alt , in ...",2025-06-11_12_11,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
7,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Gener . - Verteckung dir Gat Aloue Gomecher Gl...,2025-06-11_12_11,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
...,...,...,...,...
443,3A3Y7H67WQGOWZ55X4SWY63YD36HAF7A-ALTO10126372_...,"Aufrichtiges Heiratsgesuch . Alleins , Jungges...",2025-06-11_12_11,1 A M bih F 14 A 4 — * Ihre Kleider = Wünsche ...
444,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,Heirat . Mehrere reiche Herren in ho sichern L...,2025-06-11_12_11,Wir suchen per sofort event . 1 . Oktober Spez...
445,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,"Eabrikant der Eisenbranche , 28 Jahre alt , ka...",2025-06-11_12_11,Wir suchen per sofort event . 1 . Oktober Spez...
446,3AEACGLAAEH36KW2ZEY3PGYABF35YTHX-ALTO8167177_D...,Aufrichtiges Heiratsgesuch oder Einheirat . ic...,2025-06-11_12_11,Wir suchen per sofort event . 1 . Oktober Spez...


Laufen lassen des Strings über den Ausgangstext und extrahieren der Sequencen die eine Stringsimiliarity (Levenstein Distance) von größer 0.98 haben um zu schauen, ob wir approximativ die gleichen Texte haben.

In [42]:
dat_joined.to_pickle("dat_joined.pkl")

In [18]:
from rapidfuzz import fuzz

def find_similiar_sequence(dst_doc, source_doc, threshold:float):

    n = len(dst_doc)
    n_src = len(source_doc)
    i = 0

    best_match_list = []
    best_ratio_list = []
    while n < n_src:
        stringsim = fuzz.ratio(dst_doc, source_doc[i:n])/100
        if stringsim > threshold: 
            best_match_list.append(source_doc[i:n])
            best_ratio_list.append(stringsim)
        i += 1
        n += 1

    matches_df = pd.DataFrame(data={"ratio":best_ratio_list, "match":best_match_list, "extracted":dst_doc})
    if len(matches_df) > 0:
        return True, matches_df
    else: 
        return False
    

investigation_idx = 10
dst_doc, source_doc = not_correct_extracted_df.reset_index().text[investigation_idx], not_correct_extracted_df.reset_index().plainpagefulltext[investigation_idx]

bool_val, res_df = find_similiar_sequence(dst_doc=dst_doc, source_doc=source_doc, threshold=0.98)
res_df

,ratio,match,extracted
0,0.981447,seldorf . Ernstgemeint . Heiratsgesuch Ein jun...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
1,0.983302,eldorf . Ernstgemeint . Heiratsgesuch Ein jung...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
2,0.985158,ldorf . Ernstgemeint . Heiratsgesuch Ein junge...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
3,0.987013,dorf . Ernstgemeint . Heiratsgesuch Ein junger...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
4,0.988868,orf . Ernstgemeint . Heiratsgesuch Ein junger ...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
5,0.990724,rf . Ernstgemeint . Heiratsgesuch Ein junger M...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
6,0.992579,f . Ernstgemeint . Heiratsgesuch Ein junger Ma...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."
7,0.994434,. Ernstgemeint . Heiratsgesuch Ein junger Man...,"Ernstgemeint . Heiratsgesuch Ein junger Mann ,..."


In [19]:

def find_similiar_sequence(dst_doc:str, source_doc:str, threshold:float, return_msk:bool) -> bool:
    """Checks if the extracted content is with a certain threshold in the given document. If a 1:1 match is found it directly returns 
    True - if not levenstein distance is used to check if the extracted string is in the document with a certain threshold of stringsimiliarties
    Parameters:
        dst_doc: str - extracted content
        source_doc: str -  original document
        threshold: float - threshold for stringsimiliarities 
        return_msk: bool - parameter True - just return the mask if val is in doc or false the df with the matches and the ratio will be returned
    Returns:
        True if String is in the original document False if it is not in the document within the threshold   
     """
    
    if dst_doc in source_doc: return True
    
    n = len(dst_doc)
    n_src = len(source_doc)
    i = 0

    best_match_list = []
    best_ratio_list = []
    while n < n_src:
        stringsim = fuzz.ratio(dst_doc, source_doc[i:n])/100
        if stringsim > threshold: 
            best_match_list.append(source_doc[i:n])
            best_ratio_list.append(stringsim)
        i += 1
        n += 1

    matches_df = pd.DataFrame(data={"ratio":best_ratio_list, "match":best_match_list, "extracted":dst_doc})
    if len(matches_df) > 0 and not return_msk:
        return True, matches_df
    elif len(matches_df) > 0:
        return True
    else: 
        return False
    



In [ ]:
import pandas as pd
from rapidfuzz import fuzz

class ExtractionValidator:
    def __init__(self,dst_doc:list, source_doc:list, threshold:float ):
        self.dst_doc = dst_doc
        self.source_doc = source_doc
        self.threshold = threshold

    def find_similiar_sequence(self, dst_doc:str, source_doc:str, threshold:float) -> pd.DataFrame:
        """Checks if the extracted content is with a certain threshold in the given document. If a 1:1 match is found it directly returns 
        True - if not levenstein distance is used to check if the extracted string is in the document with a certain threshold of stringsimiliarties
        Parameters:
            dst_doc: str - extracted content
            source_doc: str -  original document
            threshold: float - threshold for stringsimiliarities 
            return_msk: bool - parameter True - just return the mask if val is in doc or false the df with the matches and the ratio will be returned
        Returns:
            pd.DataFrame containing the potential matches
        """
        
        n = len(dst_doc)
        n_src = len(source_doc)
        i = 0

        best_match_list = []
        best_ratio_list = []
        while n < n_src:
            stringsim = fuzz.ratio(dst_doc, source_doc[i:n])/100
            if stringsim > threshold: 
                best_match_list.append(source_doc[i:n])
                best_ratio_list.append(stringsim)
            i += 1
            n += 1

        matches_df = pd.DataFrame(data={"ratio":best_ratio_list, "match":best_match_list, "extracted":dst_doc})
        
        return matches_df
    
    def is_match(self, dst_doc:str, source_doc:str, threshold:float) -> bool:
        """Checks if a match was found. Check if the text is either one to one in the match or it is within the threshold similiar
        Parameters:
            dst_doc: str - extracted content
            source_doc: str -  original document
            threshold: float - threshold for stringsimiliarities 
            return_msk: bool - parameter True - just return the mask if val is in doc or false the df with the matches and the ratio will be returned
        Returns:
            True if String is in the original document False if it is not in the document within the threshold   
        
        
        """
        if dst_doc in source_doc: return True
        match_df = self.find_similiar_sequence(dst_doc, source_doc, threshold)
        if len(match_df) > 0:
            return True
        else:
            return False
        
    def apply_is_match_on_data(self) -> list[bool]:
        """Applys is_match over all data - Can be used as a mask to filter the values that are not correctly extracted
        
        Parameter:
        None

        Return: 
        List of bool values that determining if the given extracted text was is in (True) the source text within the given threshold or not (False)

        """
        found_match_list:list[bool] = [self.is_match(d, s, self.threshold) for d, s in zip(self.dst_doc, self.source_doc)]
        return found_match_list
    
    def calculate_extraction_accuracy(self) -> tuple:
        """Return tuple with the accuracy of the text extraction"""
        n = len(self.dst_doc)
        match_bool = self.apply_is_match_on_data()
        true_positive = sum(match_bool)
        
        return ("Accuracy", true_positive/n)

    
        

    

In [43]:
dat_joined = pd.read_pickle("dat_joined.pkl")

validation = ExtractionValidator(dst_doc=dat_joined.text.tolist(), source_doc=dat_joined.plainpagefulltext.tolist(), threshold=0.98)
result = validation.calculate_extraction_accuracy()
print(result)

('Accuracy is:', 0.9575892857142857)


In [40]:
len(dat_joined)

448

In [ ]:
msk = dat_joined.apply(lambda row: find_similiar_sequence(row["text"], row["plainpagefulltext"], threshold=0.95, return_msk=True), axis=1)

In [ ]:

still_not_Correct = dat_joined[msk == False]
still_not_Correct


,page_id,text,time,plainpagefulltext
135,2DW55DZZGQDFIT6CBIIFTKK73ARV7ITF-ALTO9398850_D...,"Suchen f . uns . Tochter , Is raelitin , 24 J ...",2025-06-08_20_13,Gschäffter - Verenl für Bergbau und dussstahlf...
149,2EZVCCSPNJZR5754MLRQDYIXY65QRKO5-ALTO7690695_D...,Herrscheftlüsch eingerrichtetes Haus in Barmen...,2025-06-08_20_13,Bekanntmachung . elsregister ist am in das H #...
199,2IIYBCYT6B2SO57MWJ6HQRQZCRAEWU3L-ALTO9548825_D...,"Mitte der Her . in sehr gus Verhältnissen , su...",2025-06-08_20_13,das Handelsregister ist am 14 . September 1920...
251,2LB436MW4E74OIMHD2DTFM4ISNWPPBPF-ALTO1934287_D...,"Landwirt , kath . , Junggeselle , 36 Jahre , B...",2025-06-08_20_13,"Sonntag , den 16 . September 1917 . Statt beso..."
287,2QINTW3X2TQUT2N7KW6QPCM4INRLQ5D4-ALTO9467138_D...,"Eine Gzroßbach , kath , 31 . , Istahlbandarmie...",2025-06-08_20_13,"Der Geutsche Suden . Mitdestes , fast nebelfre..."
395,373GCHWZ25DT4GNR5J44JCNUHG3GVKQX-ALTO8385016_D...,"Glückliche Heirat ! ger Herr , evgl. , großé F...",2025-06-08_20_13,An die Stammaktionäre der Howaldtswerke . Auf ...


In [ ]:
find_similiar_sequence(still_not_Correct.text[135], still_not_Correct.plainpagefulltext[135], threshold=0.75, return_msk=False)

False

In [ ]:
6/425

0.01411764705882353